In [ ]:
docs_fold = "Docs_large"
import json
import pandas as pd
import numpy as np
general_path = '~/Information Retrivial/Ass1/Docs/'+docs_fold
query_result_provided_path = general_path+'/query_res.csv'
query_result_calculated_path = general_path+'/answer.json'
query_result_calculated_csv_path = general_path+'/answer.csv'


Loading the given query result

In [ ]:
query_res = pd.read_csv(query_result_provided_path)

Reading the json file with the calculated query result

In [ ]:
answ_dict = dict()
with open(query_result_calculated_path, "r") as outfile:
    answ_dict = json.load(outfile)

answ_dict is converted in a .csv file

In [ ]:
with open(query_result_calculated_csv_path, "w") as outfile:
    outfile.write("q_num,doc_id,score")
    outfile.write('\n')
for key0 in answ_dict.keys():
    with open(query_result_calculated_csv_path, "a") as outfile:
        sub_d = answ_dict[key0]
        for key1 in sub_d.keys():
            outfile.write(str(key0)+','+str(key1)+','+str(sub_d[key1]))
            outfile.write('\n')

The created .csv file is loaded as a pandas df

In [ ]:
answer_dict = pd.read_csv(query_result_calculated_csv_path)
answer_dict = answer_dict.sort_values(['q_num', 'score'], ascending=[True, False])

In [ ]:
answer_dict

This function is used to order the result on the score

In [ ]:
def add_col_score(x, name):
    x[name] = np.arange(len(x))
    return x

The function is applied

In [ ]:
answer_dict = answer_dict.groupby(['q_num'], as_index=False).apply(lambda x: add_col_score(x, "score"))
answer_dict = answer_dict.reset_index(drop=True)

In [ ]:
answer_dict

A score column is added to query_res

In [ ]:
query_res[['q_num', 'doc_id']] = query_res
query_res = query_res[['q_num', 'doc_id']]
query_res = query_res.groupby(['q_num'], as_index=False).apply(lambda x: add_col_score(x, "col_s"))
query_res = query_res.sort_values(['q_num', 'col_s'], ascending=[True, True])
query_res

The two DF are merged to find only the queries that have the same answers in both the given solution and the calculated solution

In [ ]:
query_res = query_res.merge(answer_dict, how='inner', on = ['q_num','doc_id'])

In [ ]:
query_res

Now will be checked also if the answer are in the correct order

In [ ]:
def check_equal(x):
    if abs(x['col_s'] - x['score']) <= 10:
        return True
    return False

In [ ]:
query_res['check'] = query_res.apply(lambda x: check_equal(x), axis = 1)

In [ ]:
query_res[query_res['check'] == True]